## Baseline для хакатона Rutube по задаче «Интеллектуальный помощник оператора службы поддержки»

В рамках данного ноутбука мы рассмотрим наивный подход к решению поставленной задачи с помощью векторного поиска. Также в конце есть пример того, как поднять REST API для предоставления доступа к работающему сервису.

In [1]:
# Для запуска этого Jupyter notebook не забудьте установить необходимые пакеты:

# !pip install pandas openpyxl sentence-transformers fastapi uvicorn

In [2]:
import warnings
warnings.filterwarnings('ignore')
from collections import defaultdict

import pandas as pd

## FAQ: Часто задаваемые вопросы RUTUBE

In [3]:
# считаем базу Часто задаваемых вопросов RUTUBE 
faq = pd.read_excel("01_База_знаний.xlsx")
faq.head()

,Тема,Вопрос из БЗ,Ответ из БЗ,Классификатор 1 уровня,Классификатор 2 уровня
0,Что нельзя публиковать на RUTUBE,Что нельзя публиковать на RUTUBE?,Чужой контент без разрешения автора или правоо...,МОДЕРАЦИЯ,Отклонение/блокировка видео
1,Почему могут отключить монетизацию на видео и ...,Почему могут отключить монетизацию из-за автор...,"Монетизация может отключиться, если на вашем к...",МОНЕТИЗАЦИЯ,Отключение/подключение монетизации
2,Почему могут отключить монетизацию на видео и ...,Почему могут отключить монетизацию из-за искус...,Монетизация на RUTUBE зависит в том числе от к...,МОНЕТИЗАЦИЯ,Отключение/подключение монетизации
3,Почему могут отключить монетизацию на видео и ...,"Для каких статусов доступна монетизация, и поч...","Монетизацию на RUTUBE можно подключить, если в...",МОНЕТИЗАЦИЯ,Отключение/подключение монетизации
4,Авторское право,Какой контент можно использовать для монетизац...,"То, что вы создали сами: видео, которое вы сня...",МОНЕТИЗАЦИЯ,Отключение/подключение монетизации


## Подбор ответа из базы FAQ с помощью косинусного сходства

In [4]:
from sentence_transformers import SentenceTransformer
from sentence_transformers.util import cos_sim

# С помощью модели извлечения embbeddings из текста получим embbeddings для всех вопросов из FAQ.
# Веса модели можно найти по ссылке: https://huggingface.co/sentence-transformers/all-MiniLM-L6-v2
model = SentenceTransformer("sentence-transformers/all-MiniLM-L6-v2")
faq_embeddings = model.encode(faq['Вопрос из БЗ'].values)

In [5]:
# Извлечем embbedding для пользовательского вопроса.
question = "Как поменять пароль?"
question_embedding = model.encode(question)

# Вычислим косинусное сходство между пользовательским вопросом и каждым вопросом из FAQ.
similarities = cos_sim(question_embedding, faq_embeddings)
# Ответом будет вопрос из БЗ с максимальным косинусным сходством.
anwer_data = faq.iloc[similarities.argmax().item()]  
anwer_data.to_dict()

{'Тема': 'Общие вопросы',
 'Вопрос из БЗ': 'Как сменить пароль?',
 'Ответ из БЗ': 'Авторизуйтесь на RUTUBE, перейдите в свой профиль: https://rutube.ru/profile, нажмите «Изменить пароль» и следуйте подсказкам.',
 'Классификатор 1 уровня': 'УПРАВЛЕНИЕ АККАУНТОМ',
 'Классификатор 2 уровня': 'Персонализация'}

## Подготовка REST API

Итоговый сервис будет тестироваться через REST API тестовыми кейсами. Требования к REST API для участников таковы:

1. REST API должен принимать данные в виде следующего JSON формата: {"question": "Как изменить пароль?"}.
2. В ответ мы ожидаем данные в JSON формате: {"answer": "Какой-то ответ", "class_1": "some_class", "class_2": "some_class"}.

In [6]:
import asyncio
from fastapi import FastAPI
from pydantic import BaseModel
import uvicorn

In [7]:
class Request(BaseModel):
    question: str


class Response(BaseModel):
    answer: str
    class_1: str
    class_2: str

app = FastAPI()


@app.get("/")
def index():
    return {"text": "Интеллектуальный помощник оператора службы поддержки."}

    
@app.post("/predict")
async def predict_sentiment(request: Request):
    question_embedding = model.encode(request.question)
    similarities = cos_sim(question_embedding, faq_embeddings)
    anwer_data = faq.iloc[similarities.argmax().item()]
    response = Response(
        answer=anwer_data['Ответ из БЗ'],
        class_1=anwer_data['Классификатор 1 уровня'], # Классификатор оценивается опционально; при отсутствии можно задать константное значение.
        class_2=anwer_data['Классификатор 2 уровня'], # Классификатор оценивается опционально; при отсутствии можно задать константное значение.
    )
    return response

if __name__ == "__main__":
    host = "0.0.0.0" # Сконфигурируйте host согласно настройкам вашего сервера.
    config = uvicorn.Config(app, host=host, port=8000)
    server = uvicorn.Server(config)
    loop = asyncio.get_running_loop()
    loop.create_task(server.serve())


INFO:     Started server process [405034]
INFO:     Waiting for application startup.
INFO:     Application startup complete.
INFO:     Uvicorn running on http://0.0.0.0:8000 (Press CTRL+C to quit)


# Для проверки того, что REST API отвечает корректно, запустите данный код в отдельном ноутбуке или Python-скрипте.